# Counting number of the mutual friends

For each user having ID in the column userId count the amount of his / her common friends with each other user having ID in the column userId.

Print 49 pairs of the users having the largest amount of common friends, ordered in descending order first by the common friends count , then by id of user1 and finally by id of user 2. The format is following: "count user1 user2"7

Example:

234	54719	767867<br>
120	54719 767866<br>
97 50787 327676<br>

To solve this task use the algorithm described in the last video of lesson 1. The overall plan could look like this:

- Create a new column “friend” by exploding of column “friends” (like in the demo iPython notebook)<br>
- group the resulting dataframe by the column “friend” (like in the demo iPython notebook)<br>
- create a column “users” by collecting all users with the same id in the column “friend” together (like in the demo iPython notebook)<br>
- sort the elements in the column “users” by the function sort_array<br>
- filter only the rows which have more than 1 element in the column “users”<br>
- for each row emit all possible ordered pairs of users from the column “users” (tip: write a user defined function for this)<br>
- count the number of times each pair has appeared<br>
- with the help of the window function (like in the demo python notebook) select 49 pairs of users who have the biggest amount of common friends<br>
The sample dataset is located at /data/graphDFSample.

The part of the result on the sample dataset:


3044 21864412 51640390<br>
3021 17139850 51640390<br>
3010 14985079 51640390<br>
2970 17139850 21864412<br>
2913 20158643 27967558<br>

In [1]:
import os
execfile(os.path.join(os.environ["SPARK_HOME"], 'python/pyspark/shell.py'))

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.1.1
      /_/

Using Python version 2.7.12 (default, Nov 19 2016 06:48:10)
SparkSession available as 'spark'.


In [5]:
from pyspark.sql import SparkSession
sparkSession = SparkSession.builder.enableHiveSupport().master("local [2]").getOrCreate()

In [6]:
# data sample 
# +--------+--------------------+
# |    user|             friends|
# +--------+--------------------+
# |22991438|[20699, 175973, 5...|
# +--------+--------------------+
# assume that user 22991438 follows users 20699, 175973, 5...
# for each pair of user (A, B), count the number of users that both of them follow?
# without combine to columns
# in this case direction is important
# algorithm:
# explode friends colum to new column named friend
# now we have "user" follows "friend" in each row
# group by "friend" then collect all "user" into a list of "users"  
# now for each "friend" we have list her/his followers in column "users"
# sort user ids in "users" lists, and emit this list to all possible pair
# i.e. each pair of "users" follow "friend"
# count the number of time each pair occurs
# this will be the answer

In [7]:
graphPath = "/data/graphDFSample"

In [8]:
from pyspark.sql.functions import explode, collect_list, size, col, row_number
from pyspark.sql import Window

# Create a new column “friend” by exploding of column “friends” (like in the demo iPython notebook)
# group the resulting dataframe by the column “friend” (like in the demo iPython notebook)
# create a column “users” by collecting all users with the same id in the column “friend” together (like in the demo iPython notebook)
# sort the elements in the column “users” by the function sort_array

reversedGraph = sparkSession.read.parquet(graphPath) \
    .withColumn("friend", explode('friends')) \
    .groupBy("friend") \
    .agg(collect_list("user").alias("users"))
#     .withColumn("users_size", size("users")) \

In [10]:
# reversedGraph.take(2)

In [11]:
# sort the elements in the column “users” by the function sort_array
from pyspark.sql.functions import sort_array
reversedGraph = reversedGraph.withColumn("users", sort_array("users"))

In [12]:
# filter only the rows which have more than 1 element in the column “users”
reversedGraph = reversedGraph.filter(size('users') > 1)

In [13]:
# reversedGraph.take(2)

In [14]:
# for each row emit all possible ordered pairs of users from the column “users” (tip: write a user defined function for this)

from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, IntegerType
from itertools import combinations
# udf_friend_pair = udf(friend_pair, ArrayType)
def emit_all_pairs(users):
    return list(combinations(users, 2))

schema = ArrayType(IntegerType())
udf_emit_all_pairs = udf(emit_all_pairs, ArrayType(schema))

df = reversedGraph.withColumn("pairs", udf_emit_all_pairs(col("users")))
# df.take(2)
# [Row(friend=148, users=[3195315, 14631101, 14957568, 65051219], pairs=[[3195315, 14631101], [3195315, 14957568], [3195315, 65051219], [14631101, 14957568], [14631101, 65051219], [14957568, 65051219]])]
# emit_all_pairs([2,3,4])
# [(2, 3), (2, 4), (3, 4)]

In [15]:
# count the number of times each pair has appeared
from pyspark.sql.functions import count
df = df.select(explode("pairs").alias('pair'))\
       .withColumn('a', col("pair").getItem(0))\
       .withColumn('c', col("pair").getItem(1))\
       .groupBy('a', 'c')\
       .agg(count("*").alias("number_common_followeds"))

In [16]:
# df.show(1)

In [47]:
# with the help of the window function (like in the demo python notebook) select 49 pairs of users who have the biggest amount of common friends
# The sample dataset is located at /data/graphDFSample.
window = Window.orderBy(col("number_common_followeds").desc())
top50 = df.withColumn("row_number", row_number().over(window))\
          .filter(col("row_number") < 50) \
          .select(col('a'), col('c'), col('number_common_followeds'))\
          .orderBy(col('number_common_followeds').desc(), col('a').desc(), col('c').desc())\
          .collect()

In [6]:
# window = Window.orderBy(col("users_size").desc())
    
# top50 = reversedGraph.withColumn("row_number", row_number().over(window)) \
#             .filter(col("row_number") < 50) \
#             .select(col("friend"), col("users_size")) \
#             .orderBy(col("users_size").desc()) \
#             .collect()

In [51]:
for val in top50:
    print '%s %s %s' % (val[2], val[0], val[1])

3206 27967558 42973992
3130 20158643 42973992
3066 22582764 42973992
3044 21864412 51640390
3021 17139850 51640390
3010 14985079 51640390
2970 17139850 21864412
2913 20158643 27967558
2903 22280814 51151280
2870 23848749 51640390
2855 20158643 22582764
2849 20158643 44996025
2846 22280814 42973992
2784 21864412 23848749
2779 31964081 51640390
2776 39205988 51640390
2754 17139850 23848749
2749 22582764 27967558
2728 50561859 51640390
2724 15485897 51640390
2700 28135661 42973992
2655 22280814 27967558
2653 42973992 43548989
2639 26755857 51640390
2621 14635589 51640390
2608 15485897 17139850
2606 17139850 26755857
2601 21864412 39205988
2600 8406745 51640390
2599 37735419 51640390
2597 20158643 28135661
2585 40003405 42973992
2585 21864412 31964081
2581 27967558 43548989
2579 23848749 31964081
2578 27967558 28135661
2578 15485897 21864412
2577 42973992 64755069
2574 51151280 57077210
2573 20158643 43548989
2566 21864412 26755857
2564 22280814 64755069
2561 42973992 44996025
2556 1713985